# parse logs to create state action reward from data
see (this document for further details)[https://docs.google.com/document/d/1dwnqLLsM3YizOFxxjOR1O6U3hEZhGllhJrQLBBqrK0c/edit]

## parse log in dataframe

In [63]:
import pandas as pd
import pickle

# read log data
with open('../raw data/training-log-corpus_v2.log', 'r') as f:
    logs = f.readlines()

last_time = 0.0
last_student = ''
action_count = 0
l = []
for i, line in enumerate(logs):
    tokens = line.split('\n')[0]
    tokens = tokens.split('|')

    line_no = tokens[0]
    student_id = tokens[1]
    if len(student_id) != 8:
        print(
            f"ERROR: in line {i}, student id length should be 8, found {len(student_id)}!"
        )
        exit(0)

    action = tokens[2]

    # starting entries do not have time
    try:
        float(tokens[3])
    except:
        tokens.insert(3, 0.0)
    time = float(tokens[3])

    if last_student != student_id:
        last_time = 0.0
        # print(f"{action_count} actions found for student {last_student}")
        action_count = 0

    # check is time is almost monotonic
    if last_time - time >= 1:
        print(
            f"ERROR: in line {i}, time is not monotonic! current action {action}. total action found {action_count}. student {student_id}"
        )

    last_time = time
    last_student = student_id
    action_count += 1

    try:
        detail = tokens[4]
    except:
        detail = ""

    try:
        detail2 = tokens[5]
    except:
        detail2 = ""

    others = "|".join(tokens[6:])

    l.append({
        'line': line_no,
        'episode': student_id,
        'action': action,
        'time': time,
        'detail': detail,
        'more_detail': detail2,
        'others': others,
        'all': line.split('\n')[0]
    })

df = pd.DataFrame(l,
                  columns=['line', 'episode', 'action', 'time', 'detail', 'more_detail', 'others', 'all'])


# for students with "RESET"
# we are keeping if student reseted after planing signifcant part of the game (>70%)
# or resetted very early (<30%)
print("-- removing reset issues --")
dlt_idx = []
student_with_reset = df.loc[df['action'] == 'RESET']['episode'].unique()
for student in student_with_reset:
    df_s = df.loc[df['episode'] == student]
    total_steps = len(df_s)
    i = df.loc[(df['episode'] == student)
               & (df['action'] == 'RESET')].index[0]
    before_reset_steps = len(df_s.loc[:i])
    percent_before_reset = round(before_reset_steps / total_steps * 100, 2)
    drop_curr = []
    if percent_before_reset < 30.0:
        drop_curr = list(df_s.loc[df_s.index[0]:i + 1].index)
    elif percent_before_reset > 70.0:
        drop_curr = list(df_s.loc[i:].index)
    else:
        drop_curr = list(df_s.index)

    print(
        "student: {0} | total_steps: {1} | reset_at: {2}({3}%) | dropping: {4}"
        .format(student, total_steps, before_reset_steps, percent_before_reset,
                len(drop_curr)))
    dlt_idx += drop_curr


df = df.drop(index=dlt_idx).reset_index(drop=True)

ERROR: in line 42330, time is not monotonic! current action RESET. total action found 190. student 100-0058
ERROR: in line 62840, time is not monotonic! current action RESET. total action found 1335. student 100-0082
ERROR: in line 64365, time is not monotonic! current action RESET. total action found 755. student 100-0084
ERROR: in line 89782, time is not monotonic! current action RESET. total action found 354. student 100-0122
ERROR: in line 104788, time is not monotonic! current action RESET. total action found 501. student 100-0142
ERROR: in line 117531, time is not monotonic! current action RESET. total action found 714. student 100-0159
ERROR: in line 142386, time is not monotonic! current action RESET. total action found 348. student 100-0195
ERROR: in line 211235, time is not monotonic! current action RESET. total action found 705. student 100-0307
ERROR: in line 223610, time is not monotonic! current action RESET. total action found 1021. student 100-0322
ERROR: in line 328796

In [64]:
# get student features
df_score = pd.read_csv('../raw data/training-survey-corpus.csv')
df_score = df_score[[
    'Student ID', 'Gender', 'Game-Playing Skill', 'Content Pre Total',
    'Normalized Learning Gain'
]]
df_score = df_score.rename(
    columns={
        'Student ID': 'episode',
        'Gender': 's_static_gender',
        'Game-Playing Skill': 's_static_gameskill',
        'Content Pre Total': 's_static_pretest',
        'Normalized Learning Gain': 'nlg'
    })

# splitting reward based on median
# few nlg are None!
df_score = df_score.loc[df_score['nlg']!='None']
df_score['nlg'] = pd.to_numeric(df_score['nlg'])
mid_nlg = df_score['nlg'].describe()['50%']
df_score.loc[df_score['nlg']<mid_nlg, 'nlg'] = -100.0
df_score.loc[df_score['nlg']>=mid_nlg, 'nlg'] = 100.0

mid_pretest = df_score['s_static_pretest'].describe()['50%']
df_score.loc[df_score['s_static_pretest']<mid_pretest, 's_static_pretest'] = 0.
df_score.loc[df_score['s_static_pretest']>=mid_pretest, 's_static_pretest'] = 1.

mid_skill = df_score['s_static_gameskill'].describe()['50%']
df_score.loc[df_score['s_static_gameskill']<mid_skill, 's_static_gameskill'] = 0.
df_score.loc[df_score['s_static_gameskill']>=mid_skill, 's_static_gameskill'] = 1.

df_score['s_static_gender'] = df_score['s_static_gender'] - 1.0

# making sure we have static features for all students
# 26 students are removed | total left 399
df = df.loc[df['episode'].isin(df_score['episode'].unique())]
df = df.reset_index(drop=True)

df = df.merge(df_score, on='episode', how='left')


## parse student trajectories

### define state names and their locations for student trajectories

In [71]:
state_names = [
    # location 5 | total 5
    "s_loc_bry", "s_loc_din", "s_loc_inf", "s_loc_lab", "s_loc_men",
    # talk 8 | total 13
    "s_talk_bry", "s_talk_eli", "s_talk_ext", "s_talk_for", "s_talk_kim",
    "s_talk_que", "s_talk_rob", "s_talk_ter",
    # poster 22 | total 35
    "s_post_ali", "s_post_ant", "s_post_bac", "s_post_bac_rep", "s_post_bac_str",
    "s_post_bac_dis", "s_post_bot", "s_post_ebo", "s_post_ele_mic", "s_post_how_pat",
    "s_post_inf", "s_post_opt_mic", "s_post_pat_mut", "s_post_pre_tre", "s_post_sal",
    "s_post_sci_met", "s_post_siz_com", "s_post_sma", "s_post_vir_dis", "s_post_vir_rep",
    "s_post_vir_str", "s_post_vir",
    # book 9 | total 44
    "s_book_ant", "s_book_bac", "s_book_bot", "s_book_ebo", "s_book_fun",
    "s_book_inf", "s_book_sal", "s_book_sma", "s_book_vir",
    # objects 17 | total 61
    "s_obj_app", "s_obj_ban", "s_obj_bre", "s_obj_cac", "s_obj_che",
    "s_obj_coc", "s_obj_egg", "s_obj_mil", "s_obj_oj", "s_obj_ora",
    "s_obj_pie", "s_obj_san", "s_obj_wat", "s_obj_ket", "s_obj_jar10",
    "s_obj_jar11", "s_obj_jar4",
    # objects tested 17 | total 78
    "s_objtest_app", "s_objtest_ban", "s_objtest_bre", "s_objtest_cac", "s_objtest_che",
    "s_objtest_coc", "s_objtest_egg", "s_objtest_mil", "s_objtest_oj", "s_objtest_ora",
    "s_objtest_pie", "s_objtest_san", "s_objtest_wat", "s_objtest_ket", "s_objtest_jar10",
    "s_objtest_jar11", "s_objtest_jar4",
    # individuals | total 89
    "s_testpos", "s_testleft", "s_label", "s_label_lesson", "s_label_slide",
    "s_worksheet", "s_workshsubmit", "s_notetake", "s_noteview", "s_computer",
    "s_quiz",
    # aes 14 | total 103
    "s_aes_bry_1", "s_aes_bry_2", "s_aes_ter_1", "s_aes_ter_2", "s_aes_ter_3",
    "s_aes_kno_1", "s_aes_kno_2", "s_aes_wor_1", "s_aes_wor_2", "s_aes_wor_3",
    "s_aes_pas_1", "s_aes_pas_2", "s_aes_que_1", "s_aes_que_2",
    # player static attributes | total 106
    "s_static_gender", "s_static_pretest", "s_static_gameskill",
    # target goal | total 108
    "s_target_disease", "s_target_item"
]

action_names = [
    # location 5 | total 5
    "a_loc_bry", "a_loc_din", "a_loc_inf", "a_loc_lab", "a_loc_men",
    # talk 8 | total 13
    "a_talk_bry", "a_talk_eli", "a_talk_ext", "a_talk_for", "a_talk_kim",
    "a_talk_que", "a_talk_rob", "a_talk_ter",
    # individual 11 | total 24
    "a_obj", "a_objtest", "a_book", "a_post", "a_notetake",
    "a_noteview", "a_computer", "a_worksheet", "a_workshsubmit", "a_label",
    "a_testleft",
]

state_map = {name:i for (i,name) in enumerate(state_names)}
action_map = {name:i for (i,name) in enumerate(action_names)}
state_map_rev = {i:name for (i,name) in enumerate(state_names)}
action_map_rev = {i:name for (i,name) in enumerate(action_names)}

post_loc_map = {
    "s_post_ali": "s_loc_lab", "s_post_ant": "s_loc_inf", "s_post_bac": "s_loc_men", "s_post_bac_rep": "s_loc_men", "s_post_bac_str": "s_loc_lab",
    "s_post_bac_dis": "s_loc_lab", "s_post_bot": "s_loc_inf", "s_post_ebo": "s_loc_inf", "s_post_ele_mic": "s_loc_lab", "s_post_how_pat": "s_loc_men",
    "s_post_inf": "s_loc_inf", "s_post_opt_mic": "s_loc_lab", "s_post_pat_mut": "s_loc_lab", "s_post_pre_tre": "s_loc_inf", "s_post_sal": "s_loc_inf",
    "s_post_sci_met": "s_loc_inf", "s_post_siz_com": "s_loc_lab", "s_post_sma": "s_loc_inf", "s_post_vir_dis": "s_loc_men", "s_post_vir_rep": "s_loc_men",
    "s_post_vir_str": "s_loc_lab", "s_post_vir": "s_loc_men",
}
# all books are at s_loc_lab
talk_loc_map = {
    "s_talk_bry": "s_loc_bry",
    "s_talk_eli": "s_loc_lab",
    "s_talk_ext": "s_loc_inf",
    "s_talk_for": "s_loc_men",
    "s_talk_kim": "s_loc_inf",
    "s_talk_que": "s_loc_din",
    "s_talk_rob": "s_loc_men",
    "s_talk_ter": "s_loc_inf",
}

obj_loc_map = {
    "s_obj_app": "s_loc_din", "s_obj_ban": "s_loc_din", "s_obj_bre": "s_loc_lab", "s_obj_cac": "s_loc_bry", "s_obj_che": "s_loc_din",
    "s_obj_coc": "s_loc_lab", "s_obj_egg": "s_loc_din", "s_obj_mil": "s_loc_din", "s_obj_oj": "s_loc_din", "s_obj_ora": "s_loc_din",
    "s_obj_pie": "s_loc_din", "s_obj_san": "s_loc_din", "s_obj_wat": "s_loc_din", "s_obj_ket": "s_loc_din", "s_obj_jar10": "s_loc_inf",
    "s_obj_jar11": "s_loc_inf", "s_obj_jar4": "s_loc_inf",
}

constants = {'state_map': state_map, 'action_map': action_map, 'post_loc_map': post_loc_map, 'talk_loc_map': talk_loc_map, 'obj_loc_map': obj_loc_map, 'state_map_rev': state_map_rev, 'action_map_rev': action_map_rev}
pickle.dump(constants, open('../processed_data/constants.pkl', 'wb'))

### parse and save

In [66]:
from copy import deepcopy
import numpy as np

init_state = np.zeros(len(state_map))
init_state[state_map['s_label_lesson']] = 1  # label lesson is always available at the start

def set_loc(state, s_loc):
    # reset all location features and set to current one
    for f in [f for f in state_map.keys() if 's_loc_' in f]:
        state[state_map[f]] = 0
    state[state_map[s_loc]] = 1
    return state

all_logs = []
step = 0
episode = ""
nlg = 0.0
def add_step(state, action_name, extra=None):
    global step, episode, all_logs, nlg
    # print(episode, step, action_name, extra)
    curr_state = deepcopy(state)
    curr_action = action_map[action_name]
    next_state = deepcopy(state)
    reward = 0.0
    done = False
    if 'a_loc_' in action_name:
        val = action_name.split('a_loc_')[-1]
        next_state = set_loc(next_state, 's_loc_'+val)

    elif 'a_talk_' in action_name:
        val = action_name.split('a_talk_')[-1]
        next_state[state_map['s_talk_'+val]] = 1

    elif 'a_workshsubmit' == action_name:
        next_state[state_map["s_workshsubmit"]] += 1
        if extra is not None and extra[0] is True:
            reward = extra[1]
            done = True
            print("-- GAMEOVER WITH REWARD {0} --".format(reward))

    elif 'a_post' == action_name:
        next_state[state_map[extra]] = 1

    elif 'a_obj' == action_name:
        next_state[state_map[extra]] = 1

    elif 'a_label' == action_name:
        # labeling must be available
        if next_state[state_map["s_label_"+extra[0]]] == 1:
            next_state[state_map['s_label']] += 1
            # if solved, no longer available
            if extra[1] is True:
                next_state[state_map["s_label_"+extra[0]]] = 0
        else:
            # this action is not relevant. not logging anything
            return deepcopy(curr_state)

    elif 'a_objtest' == action_name:
        next_state[state_map['s_testleft']] -= 1
        if extra[0]!='s_objtest_nul':
            next_state[state_map[extra[0]]] = 1

        if extra[1] is True:
            next_state[state_map['s_testpos']] = 1

        if next_state[state_map['s_testleft']] < 0:
            print("ERROR! Test left is negative!")

    elif 'a_book' == action_name:
        next_state[state_map['s_book_'+extra]] = 1

    else:
        state_name = "s" + action_name[1:]
        next_state[state_map[state_name]] += 1

    all_logs.append({'episode':episode, 'step': step, 'state': curr_state,
                     'action': curr_action, 'reward': reward,
                     'next_state': next_state, 'done': done, 'nlg': nlg})
    step += 1
    return deepcopy(next_state)

skip_till_next = False
state = deepcopy(init_state)
for idx, row in df.iterrows():
    if episode != row['episode']:
        episode = row['episode']
        step = 0
        skip_till_next = False
        state = deepcopy(init_state)
        state[state_map['s_static_gender']] = row['s_static_gender']
        state[state_map['s_static_pretest']] = row['s_static_pretest']
        state[state_map['s_static_gameskill']] = row['s_static_gameskill']
        nlg = row['nlg']

    elif skip_till_next:
        continue

    if row['action']=='SOLUTION':
        disease, item = row['detail'].split("-")[1], row['more_detail'].split("-")[1]
        if disease == "Influenza":
            state[state_map['s_target_disease']] = 0
        elif disease == "Salmonellosis":
            state[state_map['s_target_disease']] = 1
        else:
            print("ERROR! Unknown target disease", disease, row['all'])
            break
        if item == "Egg":
            state[state_map['s_target_item']] = 0
        elif item == "Milk":
            state[state_map['s_target_item']] = 1
        elif item == "Sandwich":
            state[state_map['s_target_item']] = 2
        else:
            print("ERROR! Unknown target item", item, row['all'])
            break
        continue

    if row['action'] == 'ADAPTATION':
        if row['detail'] in ['select-kim-reveal', 'select-direct-toward-goal', 'select-record-finding', 'select-increase-urgency', 'select-reflection-prompt', 'select-offtask-discourage']:
            # kim redirects to see quentine. not important
            continue

        if row['detail'] == 'select-test-count':
            # this is not an aes i am tracking
            if row['others'] == 'selected-1':
                state[state_map['s_testleft']] = 3
            elif row['others'] == 'selected-2':
                state[state_map['s_testleft']] = 5
            elif row['others'] == 'selected-3':
                state[state_map['s_testleft']] = 10
            continue
        if row['detail'] == 'select-present-quiz':
            val = row['others'].split('-')[-1]
            aes = "s_aes_kno_" + val
            state[state_map[aes]] += 1
            continue
        print(row['all'])
        aes_name = row['detail'].split('-')[1][:3]
        val = row['others'].split('-')[-1]
        aes = "s_aes_" + aes_name + "_" + val
        state[state_map[aes]] += 1
        continue

    if row['action'] == "OPEN":
        curr_loc = row['more_detail'].split("cur-action-open-door-")[1][:3]  # all loc names are first 3 letters
        if curr_loc != "fri":
            state = add_step(state, 'a_loc_'+curr_loc)
        continue

    # ignore logs not relevant
    if row['action'] in ['AUTHENTICATE', 'PDAOPEN', 'PDACLOSE', 'OPEN', 'STUDY', 'ADDPOINTS', 'GOALCOMPLETE', 'NOTIFICATION', 'SELF-REPORT', 'LOOKEND', 'DROP', 'STOWITEM', 'RETRIEVEITEM', 'FINALSCORE', 'READ', 'CLOSE', 'STUDENTENTRY', 'COORGET']:
        continue

    if row['action'] == "TALK":
        who = row['more_detail'].split('cur-action-talk-')[1][:3] # all talk names are first 3 letters
        s_loc = talk_loc_map["s_talk_" + who]
        if state[state_map[s_loc]] != 1:
            state = add_step(state, 'a' + s_loc[1:])  # move to that location first
        state = add_step(state, 'a_talk_'+who)
        continue

    if row['action'] == "DIALOG":
        if row['more_detail']=="kim":
            if row['others'] == "Hmm,Iamnotsurethisisquiteright.":
                state = set_loc(state, 's_loc_inf')  # must be in the inf
                state = add_step(state, 'a_workshsubmit')
            elif row['others'] in ["Greatjob,yousolvedourmystery!", "Thisdiagnosismakesperfectsense!We'llstartthetreatmentplanrightaway."]:
                state = set_loc(state, 's_loc_inf')  # must be in the inf
                state = add_step(state, 'a_workshsubmit', extra=(True, 100.0))
                skip_till_next = True
        continue

    if row['action'] == "LOOKSTART":
        post = row['detail'].split('-')
        if len(post)>=3:
            post = "s_post_" + post[1][:3] + "_" + post[2][:3]
        elif len(post)==2:
            post = "s_post_" + post[1][:3]
        else:
            print("ERROR! wrong poster name", row['all'])
            break
        s_loc = post_loc_map[post]
        state = set_loc(state, s_loc)  # move to that location first
        state = add_step(state, 'a_post', extra=post)
        continue

    if row['action'] == "PICKUP":
        item = row['more_detail'].split('-')[-1]
        if item in ["1", "3", "null"]:
            continue
        elif item in ["10", "11", "4"]:
            item = "s_obj_jar" + item
        else:
            item = "s_obj_" + item[:3]

        s_loc = obj_loc_map[item]
        state = set_loc(state, s_loc)  # move to that location first
        state = add_step(state, 'a_obj', extra=item)
        continue

    if row['action'] == "NOTES":
        state = add_step(state, 'a_notetake')
        continue

    if row['action'] == "PDAUSE":
        if row['detail'] in ["notepad", "micromanual"]:
            state = add_step(state, 'a_noteview')
            continue
        elif row['detail'] in ['mainscreen', 'add-note', 'notify', 'take-quiz-prompt', 'pdamsg', 'pdamsg-quiz2', 'message-panel']:
            continue
        else:
            print("WHAT!", idx, row['all'])

    if row['action'] == "LABELING":
        type = row['detail']
        correct = row['others'].split("|")[-1]
        correct = [True if correct=="correct" else False]
        state = add_step(state, 'a_label', extra=(type, correct))
        continue

    if row['action'] == "TESTCOMPUTER":
        test_remaining = int(row['others'].split('|')[2].split('-')[1])
        if state[state_map['s_testleft']] != test_remaining:
            # there is a bug in adding new tests. sometimes added-test log is missed
            print("ERROR! Wrong number of test remains", row['all'])
            state[state_map['s_testleft']] = test_remaining

        state = set_loc(state, 's_loc_lab')  # must be in the lab
        state = add_step(state, 'a_loc_lab')
        continue

    if row['action'] == "TESTOBJECT":
        if row['detail'] in ["MultipleObjects", "NoObject"] or row['more_detail'] == "noenergy":
            continue

        item = row['detail'].split('-')[-1]
        if item in ["10", "11", "4"]:
            item = "s_objtest_jar" + item
        else:
            item = "s_objtest_" + item[:3]

        found = False
        if row['more_detail'] == "correct":
            found = True

        state = set_loc(state, 's_loc_lab')  # must be in the lab
        state = add_step(state, 'a_objtest', extra=(item, found))
        continue

    if row['action'] == "QUIZ":
        if row['detail'] == 'quiz-answer':
            if 'added-test' in row['others']:
                state = add_step(state, 'a_testleft')
            continue
        else:
            state[state_map['s_quiz']] += 1
            continue

    if row['action'] == "BOOKREAD":
        book = row['detail'][:3]
        state = set_loc(state, 's_loc_lab')  # move to lab first as all books are in lab
        state = add_step(state, action_name="a_book", extra=book)
        continue

    if row['action'] == "BRYCECOMPUTER":
        state = set_loc(state, 's_loc_bry')  # must be in the bryces quarters
        state = add_step(state, action_name="a_computer")
        continue

    if row['action'] == "WORKSHEET":
        if row['detail'] == "close-worksheet":
            state = add_step(state, action_name="a_worksheet")
        continue

    if row['action'] == "GAMEOVER":
        state = add_step(state, 'a_workshsubmit', extra=(True, -100.0))
        skip_till_next = True
        continue

df_logs = pd.DataFrame(all_logs)
df_logs.loc[df_logs['done']==False, 'nlg'] = 0.0
df_logs.to_pickle('../processed_data/student_trajectories_v2.pkl')

141|100-0001|ADAPTATION|738.465|select-worksheet-level|random|selected-2
168|100-0001|ADAPTATION|862.080|select-quentin-reveal|random|selected-2
202|100-0001|ADAPTATION|967.095|select-worksheet-level|random|selected-3
248|100-0001|ADAPTATION|1164.675|select-bryce-symptoms-level|random|selected-1
253|100-0001|ADAPTATION|1215.000|select-bryce-symptoms-level|random|selected-2
256|100-0001|ADAPTATION|1219.410|select-bryce-symptoms-level|random|selected-1
352|100-0001|ADAPTATION|1606.425|select-teresa-symptoms-level|random|selected-2
406|100-0001|ADAPTATION|1809.165|select-worksheet-level|random|selected-1
-- GAMEOVER WITH REWARD -100.0 --
917|100-0003|ADAPTATION|135.585|select-bryce-symptoms-level|random|selected-2
1003|100-0003|ADAPTATION|654.540|select-teresa-symptoms-level|random|selected-1
1213|100-0003|ADAPTATION|1663.290|select-quentin-reveal|random|selected-2
1528|100-0003|ADAPTATION|2928.750|select-worksheet-level|random|selected-3
1575|100-0003|ADAPTATION|3030.495|select-bryce-sym

## parse narrative trajectories

### define state action for narrative trajectories

In [67]:
def get_aes_trajectories(df, target_aes, state_map=state_map):
    aes_state_map = {fname: fid for fname, fid in state_map.items() if target_aes in fname}
    step = 0
    episode = ""
    logs = []

    state = None
    action = None
    prev_row = deepcopy(df.iloc[0])

    for i, row in df.iterrows():
        if row['episode'] != episode:
            if state is not None:
                # lazy add
                logs.append({'episode':episode, 'step': step, 'state': state, 'action': action, 'reward': prev_row['reward'], 'next_state': deepcopy(prev_row['state']), 'done': prev_row['done'], 'nlg': prev_row['nlg']})
                step = 0
                state = None

                prev_row = deepcopy(row)
            episode = row['episode']

        for fname, fid in aes_state_map.items():
            if row['state'][fid] != prev_row['state'][fid]:
                if state is not None:
                    # lazily waiting for next state
                    logs.append({'episode':episode, 'step': step, 'state': state, 'action': action, 'reward': prev_row['reward'], 'next_state': deepcopy(prev_row['state']), 'done': prev_row['done'], 'nlg': prev_row['nlg']})
                    step += 1

                state = deepcopy(prev_row['state'])
                action = int(fname.split(target_aes)[1]) - 1

        prev_row = deepcopy(row)

    if state is not None:
        # lazy add
        logs.append({'episode':episode, 'step': step, 'state': state, 'action': action, 'reward': prev_row['reward'], 'next_state': deepcopy(prev_row['state']), 'done': prev_row['done'], 'nlg': prev_row['nlg']})

    return pd.DataFrame(logs)

# gen narrative data
target_aeses = ["s_aes_bry_", "s_aes_ter_", "s_aes_que_", "s_aes_kno_", "s_aes_wor_", "s_aes_pas_"]
all_aes_trajectories = {}
for target_aes in target_aeses:
    df_aes = get_aes_trajectories(df_logs, target_aes)
    all_aes_trajectories[target_aes[2:-1]] = deepcopy(df_aes)

pickle.dump(all_aes_trajectories, open('../processed_data/narrative_trajectories_v2.pkl', 'wb'))

# random

In [86]:
np.random.choice(list(post_loc_map.keys()))

's_post_ant'

In [69]:
import gym
from gym import spaces

obs_space_tup = ()
for fname, fid in state_map.items():
    if fname in ['s_testleft', 's_label', 's_worksheet', 's_workshsubmit', 's_notetake', 's_noteview', 's_computer',
                 's_quiz'] or 's_aes_' in fname:
        obs_space_tup += (spaces.Discrete(200),)
    elif fname == 's_target_item':
        obs_space_tup += (spaces.Discrete(3),)
    else:
        obs_space_tup += (spaces.Discrete(2),)
spaces.Tuple(obs_space_tup)

len(action_map)

24